In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)
import seaborn as sns;sns.set()
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
from datetime import datetime,time


In [2]:
df=pd.read_csv(r"C:\Users\Omar\Desktop\SiTeck\Month_1\Final_Project_data\actual_data.csv")
df.head()

,week_start_dt,week_end_dt,vietnam_season,p_color,price,total_volume,brazil,india,vietnam,indonesia,china,brazil_season,indonesia_season,india_season,china_season,jordan_max_price,jordan_min_price,demand,supply
0,2015-08-17,2015-08-23,1,yellow,7.249725,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,True,True,True,True,7.425,7.025,0.5,0.5
1,2015-08-17,2015-08-23,1,red,7.600215,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,True,True,True,True,8.125,7.825,0.5,0.5
2,2015-08-17,2015-08-23,1,green,7.074805,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,True,True,True,True,7.525,7.225,0.5,0.5
3,2015-08-24,2015-08-30,1,green,7.025085,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,True,True,True,True,7.425,7.225,0.5,0.5
4,2015-08-24,2015-08-30,1,red,7.722255,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,True,True,True,True,8.125,7.825,0.5,0.5


In [3]:
df2=pd.read_csv(r"C:\Users\Omar\Desktop\SiTeck\Month_1\Final_Project_data\projection_outside_supply.csv")
df2

,week_end_dt,brazil,india,vietnam,indonesia,china,brazil_season,indonesia_season,india_season,china_season,total_volume
0,2023-10-08,62609.9,0,2365023.5,27739.8,8678.3,False,False,True,False,2464051.7
1,2023-10-15,22744.0,0,2345998.8,15052.6,6508.7,False,False,True,False,2390304.2
2,2023-10-22,10961.5,0,2555267.6,0.0,15187.1,False,False,True,False,2581416.4
3,2023-10-29,9211.5,0,2530537.9,0.0,15187.1,False,False,True,False,2554936.6


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1272 entries, 0 to 1271
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   week_start_dt     1272 non-null   object 
 1   week_end_dt       1272 non-null   object 
 2   vietnam_season    1272 non-null   int64  
 3   p_color           1272 non-null   object 
 4   price             1272 non-null   float64
 5   total_volume      1272 non-null   float64
 6   brazil            1272 non-null   float64
 7   india             1272 non-null   float64
 8   vietnam           1272 non-null   float64
 9   indonesia         1272 non-null   float64
 10  china             1272 non-null   float64
 11  brazil_season     1272 non-null   bool   
 12  indonesia_season  1272 non-null   bool   
 13  india_season      1272 non-null   bool   
 14  china_season      1272 non-null   bool   
 15  jordan_max_price  1241 non-null   float64
 16  jordan_min_price  1241 non-null   float64


In [5]:
df.interpolate(method="linear",inplace=True,limit_direction="backward")

In [6]:
df.drop(index=df.index[0:57],axis=0,inplace=True)

In [8]:
df["vietnam_season"]=df["vietnam_season"].astype("str")

In [9]:
for col in df:
    if col in ["brazil_season","indonesia_season","india_season","china_season"]:
        df[col]=df[col].apply(lambda x:1 if x is True else 0)

In [10]:
for col in df:
    if col in ["vietnam_season","p_color"]:        
        one_hot_encoded_data = pd.get_dummies(df[col])
        one_hot_encoded_data=one_hot_encoded_data.applymap(lambda x:1 if x is True else 0 )
        df=pd.concat([df.drop([col],axis=1),one_hot_encoded_data],axis=1)
        df.rename(columns={"1":"vietnam_season_1","2":"vietnam_season_2","3":"vietnam_season_3"},inplace=True)

In [11]:
from datetime import datetime,timedelta
future_date_index=pd.date_range(start="2023-10-08",end="2023-10-29",freq="W")
future_date=pd.DataFrame({"date":future_date_index}).set_index("date")

In [13]:
df=pd.concat([df,df2],axis=0)
df

,week_start_dt,week_end_dt,price,total_volume,brazil,india,vietnam,indonesia,china,brazil_season,...,jordan_max_price,jordan_min_price,demand,supply,vietnam_season_1,vietnam_season_2,vietnam_season_3,green,red,yellow
57,2015-12-28,2016-01-03,6.599075,1596040.2,10793.5,65658.1,1519588.5,0.0,0.0,0,...,6.625,6.325,0.50,0.500000,0.0,1.0,0.0,1.0,0.0,0.0
58,2015-12-28,2016-01-03,7.175335,1596040.2,10793.5,65658.1,1519588.5,0.0,0.0,0,...,7.525,7.125,0.51,0.500000,0.0,1.0,0.0,0.0,1.0,0.0
59,2015-12-28,2016-01-03,7.300575,1596040.2,10793.5,65658.1,1519588.5,0.0,0.0,0,...,7.425,7.025,16.55,0.500000,0.0,1.0,0.0,0.0,0.0,1.0
60,2016-01-04,2016-01-10,7.379675,2295578.0,5677.8,15274.4,2274625.7,0.0,0.0,0,...,7.525,7.025,271.16,0.500000,0.0,1.0,0.0,0.0,0.0,1.0
61,2016-01-04,2016-01-10,7.175335,2295578.0,5677.8,15274.4,2274625.7,0.0,0.0,0,...,7.625,7.125,42.33,0.500000,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,2023-09-25,2023-10-01,8.758362,2665343.9,167.8,45376.1,2521054.0,78334.2,8618.2,0,...,9.625,8.825,409.19,196.703982,1.0,0.0,0.0,0.0,0.0,1.0
0,NaN,2023-10-08,NaN,2464051.7,62609.9,0.0,2365023.5,27739.8,8678.3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2023-10-15,NaN,2390304.2,22744.0,0.0,2345998.8,15052.6,6508.7,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2023-10-22,NaN,2581416.4,10961.5,0.0,2555267.6,0.0,15187.1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.drop('week_start_dt',axis=1,inplace=True)

In [15]:
for col in df:
    if col in ["week_end_dt"]:
        df[col]=pd.to_datetime(df[col])
        df["year"]=df[col].dt.year
        df["month"]=df[col].dt.month

In [16]:
df.set_index("week_end_dt",inplace=True)

In [17]:
df

,price,total_volume,brazil,india,vietnam,indonesia,china,brazil_season,indonesia_season,india_season,...,demand,supply,vietnam_season_1,vietnam_season_2,vietnam_season_3,green,red,yellow,year,month
week_end_dt,,,,,,,,,,,,,,,,,,,,,
2016-01-03,6.599075,1596040.2,10793.5,65658.1,1519588.5,0.0,0.0,0,0,0,...,0.50,0.500000,0.0,1.0,0.0,1.0,0.0,0.0,2016,1
2016-01-03,7.175335,1596040.2,10793.5,65658.1,1519588.5,0.0,0.0,0,0,0,...,0.51,0.500000,0.0,1.0,0.0,0.0,1.0,0.0,2016,1
2016-01-03,7.300575,1596040.2,10793.5,65658.1,1519588.5,0.0,0.0,0,0,0,...,16.55,0.500000,0.0,1.0,0.0,0.0,0.0,1.0,2016,1
2016-01-10,7.379675,2295578.0,5677.8,15274.4,2274625.7,0.0,0.0,0,0,0,...,271.16,0.500000,0.0,1.0,0.0,0.0,0.0,1.0,2016,1
2016-01-10,7.175335,2295578.0,5677.8,15274.4,2274625.7,0.0,0.0,0,0,0,...,42.33,0.500000,0.0,1.0,0.0,0.0,1.0,0.0,2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-01,8.758362,2665343.9,167.8,45376.1,2521054.0,78334.2,8618.2,0,0,1,...,409.19,196.703982,1.0,0.0,0.0,0.0,0.0,1.0,2023,10
2023-10-08,NaN,2464051.7,62609.9,0.0,2365023.5,27739.8,8678.3,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,10
2023-10-15,NaN,2390304.2,22744.0,0.0,2345998.8,15052.6,6508.7,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023,10


In [18]:
df.fillna(0,inplace=True)

In [20]:
df

,price,total_volume,brazil,india,vietnam,indonesia,china,brazil_season,indonesia_season,india_season,...,demand,supply,vietnam_season_1,vietnam_season_2,vietnam_season_3,green,red,yellow,year,month
week_end_dt,,,,,,,,,,,,,,,,,,,,,
2016-01-03,6.599075,1596040.2,10793.5,65658.1,1519588.5,0.0,0.0,0,0,0,...,0.50,0.500000,0.0,1.0,0.0,1.0,0.0,0.0,2016,1
2016-01-03,7.175335,1596040.2,10793.5,65658.1,1519588.5,0.0,0.0,0,0,0,...,0.51,0.500000,0.0,1.0,0.0,0.0,1.0,0.0,2016,1
2016-01-03,7.300575,1596040.2,10793.5,65658.1,1519588.5,0.0,0.0,0,0,0,...,16.55,0.500000,0.0,1.0,0.0,0.0,0.0,1.0,2016,1
2016-01-10,7.379675,2295578.0,5677.8,15274.4,2274625.7,0.0,0.0,0,0,0,...,271.16,0.500000,0.0,1.0,0.0,0.0,0.0,1.0,2016,1
2016-01-10,7.175335,2295578.0,5677.8,15274.4,2274625.7,0.0,0.0,0,0,0,...,42.33,0.500000,0.0,1.0,0.0,0.0,1.0,0.0,2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-01,8.758362,2665343.9,167.8,45376.1,2521054.0,78334.2,8618.2,0,0,1,...,409.19,196.703982,1.0,0.0,0.0,0.0,0.0,1.0,2023,10
2023-10-08,0.000000,2464051.7,62609.9,0.0,2365023.5,27739.8,8678.3,0,0,1,...,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2023,10
2023-10-15,0.000000,2390304.2,22744.0,0.0,2345998.8,15052.6,6508.7,0,0,1,...,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2023,10
